This is the notebook related to HMM data preprocessing. To run this, please uncomment all the cells an press 'Run all' button.

In [31]:
# Set the path and input parameters
import os
directory = os.getcwd() # the main directory of the project

# The credentials for the remote cluster
name = 'alina'
server = 'ecate'

In [32]:
# Importing the libraries
from functions import *

## HMM: Data preparation
In this part we generate HMM models based on MSA and retrieve the results from hmmsearch and then from Pfam. We start with the loading of the `disordered` dataframe and analyse how many proteins and disordered regions we have.

In [33]:
# Set the maximum width of the columns
pd.set_option('display.max_colwidth', 20)

In [34]:
# Load the disordered and curated_disprot dataframes
disordered_df = pd.read_csv('{}/disordered_df.csv'.format(directory))
curated_disprot_df = pd.read_csv('{}/curated_disprot.csv'.format(directory))

print('The number of rows with the disordered regions: {}'.format(len(disordered_df)))

The number of rows with the disordered regions: 7393


In [35]:
# Collect the disordered regions in a dictionary
dis_regs = set()

for i, row in disordered_df.iterrows():
    dis_id = row[0]
    matching_row = curated_disprot_df[curated_disprot_df['acc'] == dis_id]
    if not matching_row.empty:
        region = matching_row['region']
        dis_regs.update(region)
        dis_regs_list = list(dis_regs) # convert set to a list
        
# Define an array of disordered regions ids
disprot_ids = disordered_df['query_id'].unique()

print('The number of proteins with disordered regions: {}'.format(len(disprot_ids)))
print('The number of disordered regions in the proteins: {}'.format(len(dis_regs_list)))

The number of proteins with disordered regions: 39
The number of disordered regions in the proteins: 53


The results above show how many proteins with the disordered regions are in the provided data, as well as how many disordered regions do they contain.

## 1. hmmbuild
We build an HMM of each disordered region, using trimmed MSA as an input. We will build the models separately for BLAST and ClustalOmega to be able to compare the quality of the models based on the local and the global alignment.

In [67]:
# Set the paths to MSA (input) and HMM (output) files - BLAST
msa_dir_blast = '{}/results/alignments/output_files/disordered/blast'.format(directory)
hmmbuild_dir_blast = '{}/results/hmms/hmmbuild/blast'.format(directory)

# Create directories if they don't exist
create_directory(msa_dir_blast)
create_directory(hmmbuild_dir_blast)

# Set the paths to  MSA (input) and HMM (output) files - ClustalOmega
msa_dir_clustal = '{}/results/alignments/output_files/disordered/clustal'.format(directory)
hmmbuild_dir_clustal = '{}/results/hmms/hmmbuild/clustal'.format(directory)

# Create directories if they don't exist
create_directory(msa_dir_clustal)
create_directory(hmmbuild_dir_clustal)

Directory /Users/alina/HMM/results/alignments/output_files/disordered/blast already exists.
Directory /Users/alina/HMM/results/hmms/hmmbuild/blast already exists.
Directory /Users/alina/HMM/results/alignments/output_files/disordered/clustal already exists.
Directory /Users/alina/HMM/results/hmms/hmmbuild/clustal already exists.


In [37]:
# # Build HMM for BLAST
# for filename in os.listdir(msa_dir_blast):
#     if filename.endswith('.fasta'):
#         input_file = os.path.join(msa_dir_blast, filename)
#         output_file = os.path.join(hmmbuild_dir_blast, os.path.splitext(filename)[0] + '.hmm')

#         subprocess.run(['hmmbuild', output_file, input_file])
#         print('hmmbuild completed for {}'.format(filename))

In [38]:
# # Build HMM for ClustalOmega
# for filename in os.listdir(msa_dir_clustal):
#     if filename.endswith('.fasta'):
#         input_file = os.path.join(msa_dir_clustal, filename)
#         output_file = os.path.join(hmmbuild_dir_clustal, os.path.splitext(filename)[0] + '.hmm')

#         subprocess.run(['hmmbuild', output_file, input_file])
#         print('hmmbuild completed for {}'.format(filename))

## 2. hmmsearch

After building the models, our objective is to assess if overlaps with the profiles in the sequence database (Reference Proteome 75%) exist. We retrive the data containing the most significant sequences, with a default E-value threshold of 0.01.

The same as before, we do it for BLAST and ClustalOmega separately.

In [39]:
# # Copy the HMMs from the BLAST folder
# for filename in os.listdir(hmmbuild_dir_blast):
#     source_path = os.path.join(hmmbuild_dir_blast, filename)
#     destination_path = '{}@{}:~/hmm/blast/hmmbuild/{}'.format(name, server, filename)
#     subprocess.run(['scp', source_path, destination_path])
#     print('File {} copied to {}'.format(filename, destination_path))

In [40]:
# # Copy the HMMs from the ClustalOmega folder
# for filename in os.listdir(hmmbuild_dir_clustal):
#     source_path = os.path.join(hmmbuild_dir_clustal, filename)
#     destination_path = '{}@{}:~/hmm/clustal/hmmbuild/{}'.format(name, server, filename)
#     subprocess.run(['scp', source_path, destination_path])
#     print('File {} copied to {}'.format(filename, destination_path))

After copying the files, we run `hmmsearch` command. 
It is done remotely by running the command `sbatch start_array_jobs.sh <ali_type>`.
`<ali_type>` can be either `blast` or `clustal`.

In [69]:
# Construct the source and the target directories for BLAST - remote computer
blast_in_dir_rem = '/home/{}/hmm/blast/hmmbuild'.format(name)
blast_out_dir_rem = '/home/{}/hmm/blast/hmmsearch'.format(name)

# # List files in the source directory on the remote machine
# ls_command_blast = 'ssh {}@{} "ls {}"'.format(name, server, blast_in_dir_rem)
# file_list_blast = get_ipython().getoutput(ls_command_blast)

# Then run sbatch start_array_jobs.sh blast on the remote computer

In [70]:
# # # Copy BLAST results to the local folder
# hmmsearch_dir_blast = '{}/results/hmms/hmmsearch/blast'.format(directory)

# for filename in file_list_blast:
#     filename = filename.strip()
#     source_path = '{}/{}'.format(blast_in_dir_rem, filename)
#     dest_filename = os.path.splitext(filename)[0]
    
#     # Copy files
#     !scp {name}@{server}:~/hmm/blast/hmmsearch/hmmsearch_blast_{dest_filename}.txt {hmmsearch_dir_blast}

In [71]:
# Construct the source and the target directories for ClustalOmega
clustal_in_dir_rem = '/home/{}/hmm/clustal/hmmbuild'.format(name)
clustal_out_dir_rem = '/home/{}/hmm/clustal/hmmsearch'.format(name)

# # List files in the source directory on the remote machine
# ls_command_clustal = 'ssh {}@{} "ls {}"'.format(name, server, clustal_in_dir_rem)
# file_list_clustal = get_ipython().getoutput(ls_command_clustal)

# Then run sbatch start_array_jobs.sh clustal on the remote computer

In [72]:
# # # Copy ClustalOmega results to the local folder
# hmmsearch_dir_clustal = '{}/results/hmms/hmmsearch/clustal'.format(directory)

# for filename in file_list_clustal:
#     filename = filename.strip()
#     source_path = '{}/{}'.format(clustal_in_dir_rem, filename)
#     dest_filename = os.path.splitext(filename)[0]
    
#     # Copy files
#     !scp {name}@{server}:~/hmm/clustal/hmmsearch/hmmsearch_clustal_{dest_filename}.txt {hmmsearch_dir_clustal}

In [73]:
# Preprocess hmmsearch results - BLAST
hmmsearch_dir_blast = '{}/results/hmms/hmmsearch/blast'.format(directory)
hmmsearch_dir_blast_prepr_output = '{}/results/hmms/hmmsearch/blast/hmmsearch_df'.format(directory)
create_directory(hmmsearch_dir_blast)
create_directory(hmmsearch_dir_blast_prepr_output)

blast_results_list = []

for filename in os.listdir(hmmsearch_dir_blast):
    if filename.endswith('.txt'):
        file_path = os.path.join(hmmsearch_dir_blast, filename)
        parts = filename.split('_')
        protein_id = '{}_{}'.format(parts[2], parts[3].split(".")[0])
        try:
            blast_stat = process_hmmsearch_file(file_path) 
            blast_reg = extract_table_from_output(file_path)
            blast_result = pd.merge(blast_stat, blast_reg, left_on='Sequence', right_on='id', how='inner')
            blast_result = blast_result.drop(columns=['Description', 'id'])
            
            # Save individual DataFrames to a list
            blast_results_list.append(blast_result)
        
            # Save the DataFrame to a CSV file
            output_file = os.path.join(hmmsearch_dir_blast_prepr_output, 'hmmsearch_blast_df_{}.csv'.format(protein_id))
            blast_result.to_csv(output_file, index=False)
            
        except ValueError as e:
            print('Error processing file {}: {}'.format(filename, e))
            
# Merge all DataFrames into one
hmmsearch_results_blast = pd.concat(blast_results_list, ignore_index=True)

# Save the merged DataFrame to a CSV file
merged_output_file = os.path.join('{}/results/hmms/hmmsearch/'.format(directory), 'hmmsearch_results_blast.csv')
hmmsearch_results_blast.to_csv(merged_output_file, index=False)
print('The number of retrieved hmmsearch results for BLAST: {}'.format(len(hmmsearch_results_blast)))
hmmsearch_results_blast.head()

Directory /Users/alina/HMM/results/hmms/hmmsearch/blast already exists.
Directory /Users/alina/HMM/results/hmms/hmmsearch/blast/hmmsearch_df already exists.
Error processing file hmmsearch_blast_O15922_230-240.txt: '  ------ inclusion threshold ------\n' is not in list
Error processing file hmmsearch_blast_O15922_315-324.txt: '  ------ inclusion threshold ------\n' is not in list
Error processing file hmmsearch_blast_J8TM36_236-249.txt: '  ------ inclusion threshold ------\n' is not in list
Error processing file hmmsearch_blast_Q5T4W7_108-120.txt: '  ------ inclusion threshold ------\n' is not in list
The number of retrieved hmmsearch results for BLAST: 189624


,E-value,score,bias,E-value,score,bias,exp,N,Sequence,hmm_from,hmm_to,hmm_length,ali_from,ali_to,ali_length,env_from,env_to,env_length
0,0,5480.3,5662.4,3.8e-05,35.9,8.9,378.4,366,A0A7M7NU54,1,35,35,246,282,37,246,284,39
1,0,2495.1,2437.9,4.4e-05,35.7,5.8,148.9,146,A0A7I8W901,1,34,34,273,306,34,273,309,37
2,0,2055.8,1958.2,1e-05,37.7,8.0,138.2,129,A0A8B7ZVU6,1,34,34,345,378,34,345,381,37
3,0,2040.0,1932.9,1e-05,37.7,8.0,136.2,128,A0A8B7ZRF1,1,32,32,382,413,32,382,416,35
4,0,1770.3,1740.7,1.1e-05,37.6,9.4,122.3,113,A0A8B7ZPY7,1,34,34,345,378,34,345,381,37


In [74]:
# Preprocess hmmsearch results - ClustalOmega
hmmsearch_dir_clustal = '{}/results/hmms/hmmsearch/clustal'.format(directory)
hmmsearch_dir_clustal_prepr_output = '{}/results/hmms/hmmsearch/clustal/hmmsearch_df'.format(directory)
create_directory(hmmsearch_dir_clustal)
create_directory(hmmsearch_dir_clustal_prepr_output)

clustal_results_list = []

for filename in os.listdir(hmmsearch_dir_clustal):
    if filename.endswith('.txt'):
        file_path = os.path.join(hmmsearch_dir_clustal, filename)
        parts = filename.split('_')
        protein_id = '{}_{}'.format(parts[2], parts[3].split(".")[0])
        try:
            clustal_stat = process_hmmsearch_file(file_path) 
            clustal_reg = extract_table_from_output(file_path)
            clustal_result = pd.merge(clustal_stat, clustal_reg, left_on='Sequence', right_on='id', how='inner')
            clustal_result = clustal_result.drop(columns=['Description', 'id'])
            
            # Save individual DataFrames to a list
            clustal_results_list.append(clustal_result)
        
            # Save the DataFrame to a CSV file
            output_file = os.path.join(hmmsearch_dir_clustal_prepr_output, 'hmmsearch_clustal_df_{}.csv'.format(protein_id))
            clustal_result.to_csv(output_file, index=False)
            
        except ValueError as e:
            print('Error processing file {}: {}'.format(filename, e))
            
# Merge all DataFrames into one
hmmsearch_results_clustal = pd.concat(clustal_results_list, ignore_index=True)

# Save the merged DataFrame to a CSV file
merged_output_file = os.path.join('{}/results/hmms/hmmsearch/'.format(directory), 'hmmsearch_results_clustal.csv')
hmmsearch_results_clustal.to_csv(merged_output_file, index=False)
print('The number of retrieved hmmsearch results for ClustalOmega: {}'.format(len(hmmsearch_results_clustal)))
hmmsearch_results_clustal.head()

Directory /Users/alina/HMM/results/hmms/hmmsearch/clustal already exists.
Directory /Users/alina/HMM/results/hmms/hmmsearch/clustal/hmmsearch_df already exists.
Error processing file hmmsearch_clustal_J8TM36_236-249.txt: '  ------ inclusion threshold ------\n' is not in list
Error processing file hmmsearch_clustal_O15922_315-324.txt: '  ------ inclusion threshold ------\n' is not in list
Error processing file hmmsearch_clustal_Q5T4W7_108-120.txt: '  ------ inclusion threshold ------\n' is not in list
The number of retrieved hmmsearch results for ClustalOmega: 211312


,E-value,score,bias,E-value,score,bias,exp,N,Sequence,hmm_from,hmm_to,hmm_length,ali_from,ali_to,ali_length,env_from,env_to,env_length
0,3.8e-32,122.0,13.1,4.4e-32,121.8,13.1,1.0,1,A0A8C7BFT3,1,61,61,9,69,61,9,70,62
1,2.4e-31,119.5,13.6,9.9e-31,117.5,13.6,2.2,1,A0A3Q7NSQ2,1,62,62,352,413,62,352,413,62
2,3e-31,119.2,13.6,1.4e-30,117.0,13.6,2.3,1,A0A2U3ZCX9,1,62,62,451,512,62,451,512,62
3,3e-31,119.1,13.6,1.4e-30,117.0,13.6,2.2,1,A0A2U3VQT6,1,62,62,454,515,62,454,515,62
4,3.1e-31,119.1,13.6,1.4e-30,117.0,13.6,2.2,1,A0A3Q7P1S3,1,62,62,451,512,62,451,512,62


We can see that 4 (3 for ClustalOmega) regions of proteins do not have HMM matches in RP 75% database: 

**O15922_230-240** - wasn't found only for BLAST;

**O15922_315-324, J8TM36_236-249, Q5T4W7_108-120** - for both BLAST and ClustalOmega.

The possible reason may lie in the size of the region - all of them are quite small, about 10-14 amino acids. By analysing other short disordered regions like **O43791_169-178** (the length of 10 amino acids) we can see that there are reported the results only below the threshold.

In terms of region we are interested in the 3 fields:
- `hmm_from`-`hmm_to`: the endpoints of the hmm profile. In our case the beginning is usually 1, since we parse not the initial alignment as an input, but the separate disordered regions where the position mostly starts from 1 (in case there are no gaps in the subject sequences).
- `ali_from`-`ali_to`: the endpoints of the target sequence. They are obtained by matching HMM to the sequence, thus obtaining the positions.
- `env_from`-`env_to`: the envelope of the domain's location. It's usually a bit wider than the alignment.

## 3. Pfam
In this part we will use the proteins obtained in the dataframes with the `hmmsearch` results to iterate over the Protein2ipr database to find which HMM models do already exist in Pfam.

In [75]:
# # Copy the files with the hmmsearch statistics to the remote computer
# !scp {directory}/results/hmms/hmmsearch/hmmsearch_results_blast.csv {name}@{server}:~/stats/hmmsearch_results_blast.csv
# !scp {directory}/results/hmms/hmmsearch/hmmsearch_results_clustal.csv {name}@{server}:~/stats/hmmsearch_results_clustal.csv

The file `filtered.tsv.gz` is a shorter archive derived from `protein2ipr` database. **It contains only the domains from Interpro**. We check the overlap of the found results from `hmmsearch`.

In [76]:
# # Check the overlaps with Interpro domains (BLAST)
# !ssh {name}@{server} '/home/alina/protein2ipr.py /home/alina/stats/blast /home/alina/filtered.tsv.gz protein2ipr_blast.tsv'

In [77]:
# # Copy the files to the local folder
# !scp {name}@{server}:~/protein2ipr_blast.tsv {directory}/results/pfam

In [78]:
# # Check the overlaps with Interpro domains (ClustalOmega)
# !ssh {name}@{server} '/home/alina/protein2ipr.py /home/alina/stats/clustal /home/alina/filtered.tsv.gz protein2ipr_clustal.tsv'

In [79]:
# # Copy the files to the local folder
# !scp {name}@{server}:~/protein2ipr_clustal.tsv {directory}/results/pfam

## 4. Results preprocessing

In [80]:
# Filter only entries with Pfam ID and intercepting regions with the curated_disprot instances - BLAST
filename_blast = '{}/results/pfam/protein2ipr_blast.tsv'.format(directory)
pfam_blast = pfam_processing(filename_blast)

print('The number of retrieved Pfam instances for BLAST: {}'.format(len(pfam_blast)))
pfam_blast.head()

The number of retrieved Pfam instances for BLAST: 927771


,uniprot_id,pfam_id,ipr_id,start_pfam,end_pfam,length_pfam
0,A0A010Q304,PF02775,IPR011766,499,646,148
1,A0A010Q304,PF00205,IPR012000,289,434,146
2,A0A010Q304,PF02776,IPR012001,91,205,115
3,A0A010Q7P7,PF00018,IPR001452,435,481,47
4,A0A010Q7P7,PF03114,IPR004148,113,226,114


In [81]:
# Filter only entries with Pfam ID and intercepting regions with the curated_disprot instances - ClustalOmega
filename_clustal = '{}/results/pfam/protein2ipr_clustal.tsv'.format(directory)
pfam_clustal = pfam_processing(filename_clustal)

print('The number of retrieved Pfam instances for ClustalOmega: {}'.format(len(pfam_clustal)))
pfam_clustal.head()

The number of retrieved Pfam instances for ClustalOmega: 1073417


,uniprot_id,pfam_id,ipr_id,start_pfam,end_pfam,length_pfam
0,A0A010Q304,PF02775,IPR011766,499,646,148
1,A0A010Q304,PF00205,IPR012000,289,434,146
2,A0A010Q304,PF02776,IPR012001,91,205,115
3,A0A010Q7P7,PF00018,IPR001452,435,481,47
4,A0A010Q7P7,PF03114,IPR004148,113,226,114


In [82]:
# Merge hmmsearch results with Pfam - BLAST
pfam_blast = pd.merge(pfam_blast, 
                      hmmsearch_results_blast[['Sequence', 
                                               'hmm_from', 'hmm_to', 'hmm_length',
                                               'ali_from', 'ali_to', 'ali_length',
                                               'env_from', 'env_to', 'env_length']], 
                      left_on='uniprot_id', right_on='Sequence', how='left')

pfam_blast = pfam_blast.dropna(axis=0)
pfam_blast = pfam_blast.drop(columns='Sequence')
print('The number of retrieved Pfam instances for BLAST: {}'.format(len(pfam_blast)))
pfam_blast.head()

The number of retrieved Pfam instances for BLAST: 950184


,uniprot_id,pfam_id,ipr_id,start_pfam,end_pfam,length_pfam,hmm_from,hmm_to,hmm_length,ali_from,ali_to,ali_length,env_from,env_to,env_length
0,A0A010Q304,PF02775,IPR011766,499,646,148,1,41,41,650,690,41,650,690,41
1,A0A010Q304,PF02775,IPR011766,499,646,148,1,16,16,583,598,16,583,598,16
2,A0A010Q304,PF00205,IPR012000,289,434,146,1,41,41,650,690,41,650,690,41
3,A0A010Q304,PF00205,IPR012000,289,434,146,1,16,16,583,598,16,583,598,16
4,A0A010Q304,PF02776,IPR012001,91,205,115,1,41,41,650,690,41,650,690,41


In [83]:
# Merge hmmsearch results with Pfam - ClustalOmega
pfam_clustal = pd.merge(pfam_clustal, 
                        hmmsearch_results_clustal[['Sequence', 
                                                   'hmm_from', 'hmm_to', 'hmm_length',
                                                   'ali_from', 'ali_to', 'ali_length',
                                                   'env_from', 'env_to', 'env_length']], 
                        left_on='uniprot_id', right_on='Sequence', how='left')

pfam_clustal = pfam_clustal.dropna(axis=0)
pfam_clustal = pfam_clustal.drop(columns='Sequence')
print('The number of retrieved Pfam instances for ClustalOmega: {}'.format(len(pfam_clustal)))
pfam_clustal.head()

The number of retrieved Pfam instances for ClustalOmega: 1110208


,uniprot_id,pfam_id,ipr_id,start_pfam,end_pfam,length_pfam,hmm_from,hmm_to,hmm_length,ali_from,ali_to,ali_length,env_from,env_to,env_length
0,A0A010Q304,PF02775,IPR011766,499,646,148,1,16,16,583,598,16,583,598,16
1,A0A010Q304,PF02775,IPR011766,499,646,148,1,40,40,650,689,40,650,690,41
2,A0A010Q304,PF00205,IPR012000,289,434,146,1,16,16,583,598,16,583,598,16
3,A0A010Q304,PF00205,IPR012000,289,434,146,1,40,40,650,689,40,650,690,41
4,A0A010Q304,PF02776,IPR012001,91,205,115,1,16,16,583,598,16,583,598,16


Then we add the calculated overlaps in the dataframe using the function `pfam_hmm_overlap`.

- `overl_len`: the common region of the DisProt-HMM and Pfam-HMM. We find the minimum of the end position both in Pfam and HMM and the maximum of start position. Their difference will give us the overlap.
- `overlap_pfam`: % of overlap with Pfam only. We divide `overl_len` by the length of Pfam.
- `overlap_ali`: % of overlap with HMM only. We divide `overl_len` by the length of HMM.
- `non_overlap_len`: the non-covered region, the rest of the Pfam and HMM regions.
- `overlap_perc`: % of covered region. Calculated by division of `overl_len` to the whole length of both Pfam and HMM.

In [84]:
# Function to calculate overlaps between DisProt-HMM and Pfam-HMM
def pfam_hmm_overlap(row_pfam):
    # Calculate the overlap of hmm (ali in this case) and the Pfam
    overlap_len = min(row_pfam['end_pfam'], row_pfam['ali_to']) - max(row_pfam['start_pfam'], 
                                                                      row_pfam['ali_from']) + 1 
    overlap_pfam = overlap_len / row_pfam['length_pfam'] * 100 # % of length overlap with Pfam
    overlap_ali = overlap_len / row_pfam['ali_length'] * 100 # % of length overlap with HMM (ali)
    max_length = row_pfam['length_pfam'] + row_pfam['ali_length'] - overlap_len # the whole region covered by hmm and pfam
    non_overlap_len = max_length - overlap_len + 1 # non covered region

    if overlap_len > 0:
        overlap_perc = (overlap_len / max_length) * 100 
        overlap_sym = 2 * overlap_len / (row_pfam['ali_length'] + row_pfam['length_pfam'])
    else:
        overlap_len = 0
        overlap_pfam = 0
        overlap_ali = 0
        overlap_perc = 0
        overlap_sym = 0

    return overlap_len, overlap_pfam, overlap_ali, non_overlap_len, overlap_perc, overlap_sym

In [85]:
# Add the overlaps to the dataframe - BLAST
overlap_pfam_hmm = []

for index_pfam, row_pfam in pfam_blast.iterrows():
    overl_len, overl_pfam, overl_ali, non_overl_len, overl_perc, overlap_sym = pfam_hmm_overlap(row_pfam)
    overl_pfam = round(overl_pfam, 2)
    overl_ali = round(overl_ali, 2)
    overl_perc = round(overl_perc, 2)
    overlap_sym = round(overlap_sym, 2)
    overlap_pfam_hmm.append((overl_len, overl_pfam, overl_ali, non_overl_len, overl_perc, overlap_sym))

# Extract overlap_pfam and overlap_hmm from the list of tuples
pfam_blast['overl_len'], pfam_blast['overl_pfam'], pfam_blast['overl_ali'], pfam_blast['non_overl_len'], pfam_blast['overl_perc'], pfam_blast['overlap_sym'] = zip(*overlap_pfam_hmm)
pfam_blast.to_csv(f'results/pfam/pfam_overlap/pfam_blast.csv', index=False)
pfam_blast.head()

,uniprot_id,pfam_id,ipr_id,start_pfam,end_pfam,length_pfam,hmm_from,hmm_to,hmm_length,ali_from,...,ali_length,env_from,env_to,env_length,overl_len,overl_pfam,overl_ali,non_overl_len,overl_perc,overlap_sym
0,A0A010Q304,PF02775,IPR011766,499,646,148,1,41,41,650,...,41,650,690,41,0,0.00,0.0,196,0.00,0.0
1,A0A010Q304,PF02775,IPR011766,499,646,148,1,16,16,583,...,16,583,598,16,16,10.81,100.0,133,10.81,0.2
2,A0A010Q304,PF00205,IPR012000,289,434,146,1,41,41,650,...,41,650,690,41,0,0.00,0.0,618,0.00,0.0
3,A0A010Q304,PF00205,IPR012000,289,434,146,1,16,16,583,...,16,583,598,16,0,0.00,0.0,459,0.00,0.0
4,A0A010Q304,PF02776,IPR012001,91,205,115,1,41,41,650,...,41,650,690,41,0,0.00,0.0,1045,0.00,0.0


In [86]:
pfam_blast[pfam_blast['overl_len'] != 0]

,uniprot_id,pfam_id,ipr_id,start_pfam,end_pfam,length_pfam,hmm_from,hmm_to,hmm_length,ali_from,...,ali_length,env_from,env_to,env_length,overl_len,overl_pfam,overl_ali,non_overl_len,overl_perc,overlap_sym
1,A0A010Q304,PF02775,IPR011766,499,646,148,1,16,16,583,...,16,583,598,16,16,10.81,100.00,133,10.81,0.20
6,A0A010Q7P7,PF00018,IPR001452,435,481,47,10,65,56,428,...,58,424,487,64,47,100.00,81.03,12,81.03,0.90
10,A0A010QT56,PF00018,IPR001452,707,751,45,17,67,51,707,...,51,700,758,59,45,100.00,88.24,7,88.24,0.94
14,A0A010R8W1,PF00018,IPR001452,956,1004,49,17,65,49,957,...,52,954,1010,57,48,97.96,92.31,6,90.57,0.95
15,A0A010RFU4,PF00018,IPR001452,229,273,45,12,68,57,225,...,57,218,284,67,45,100.00,78.95,13,78.95,0.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952069,X6N6Z8,PF07653,IPR001452,2,33,32,34,64,31,2,...,31,1,39,39,31,96.88,100.00,2,96.88,0.98
952070,X6N7N7,PF00018,IPR001452,61,107,47,10,66,57,55,...,58,52,117,66,47,100.00,81.03,12,81.03,0.90
952073,X6NHX1,PF00018,IPR001452,41,85,45,6,66,61,30,...,61,25,94,70,45,100.00,73.77,17,73.77,0.85
952074,X6NKT1,PF14604,IPR001452,10,56,47,15,69,55,6,...,55,4,62,59,47,100.00,85.45,9,85.45,0.92


In [87]:
# Add the overlaps to the dataframe - ClustalOmega
overlap_pfam_hmm = []

for index_pfam, row_pfam in pfam_clustal.iterrows():
    overl_len, overl_pfam, overl_ali, non_overl_len, overl_perc, overlap_sym = pfam_hmm_overlap(row_pfam)
    overl_pfam = round(overl_pfam, 2)
    overl_ali = round(overl_ali, 2)
    overl_perc = round(overl_perc, 2)
    overlap_sym = round(overlap_sym, 2)
    overlap_pfam_hmm.append((overl_len, overl_pfam, overl_ali, non_overl_len, overl_perc, overlap_sym))

# Extract overlap_pfam and overlap_hmm from the list of tuples
pfam_clustal['overl_len'], pfam_clustal['overl_pfam'], pfam_clustal['overl_ali'], pfam_clustal['non_overl_len'], pfam_clustal['overl_perc'], pfam_clustal['overlap_sym'] = zip(*overlap_pfam_hmm)
pfam_clustal.to_csv(f'results/pfam/pfam_overlap/pfam_clustal.csv', index=False)
pfam_clustal.head()

,uniprot_id,pfam_id,ipr_id,start_pfam,end_pfam,length_pfam,hmm_from,hmm_to,hmm_length,ali_from,...,ali_length,env_from,env_to,env_length,overl_len,overl_pfam,overl_ali,non_overl_len,overl_perc,overlap_sym
0,A0A010Q304,PF02775,IPR011766,499,646,148,1,16,16,583,...,16,583,598,16,16,10.81,100.0,133,10.81,0.2
1,A0A010Q304,PF02775,IPR011766,499,646,148,1,40,40,650,...,40,650,690,41,0,0.00,0.0,195,0.00,0.0
2,A0A010Q304,PF00205,IPR012000,289,434,146,1,16,16,583,...,16,583,598,16,0,0.00,0.0,459,0.00,0.0
3,A0A010Q304,PF00205,IPR012000,289,434,146,1,40,40,650,...,40,650,690,41,0,0.00,0.0,617,0.00,0.0
4,A0A010Q304,PF02776,IPR012001,91,205,115,1,16,16,583,...,16,583,598,16,0,0.00,0.0,886,0.00,0.0


In [88]:
pfam_clustal[pfam_clustal['overl_len'] != 0]

,uniprot_id,pfam_id,ipr_id,start_pfam,end_pfam,length_pfam,hmm_from,hmm_to,hmm_length,ali_from,...,ali_length,env_from,env_to,env_length,overl_len,overl_pfam,overl_ali,non_overl_len,overl_perc,overlap_sym
0,A0A010Q304,PF02775,IPR011766,499,646,148,1,16,16,583,...,16,583,598,16,16,10.81,100.00,133,10.81,0.20
6,A0A010Q7P7,PF00018,IPR001452,435,481,47,10,65,56,428,...,58,424,487,64,47,100.00,81.03,12,81.03,0.90
8,A0A010QBJ0,PF00018,IPR001452,1080,1125,46,18,68,51,1082,...,51,1073,1135,63,44,95.65,86.27,10,83.02,0.91
11,A0A010QT56,PF00018,IPR001452,707,751,45,16,66,51,706,...,51,694,758,65,45,100.00,88.24,7,88.24,0.94
15,A0A010R8W1,PF00018,IPR001452,956,1004,49,19,66,48,959,...,51,950,1010,61,46,93.88,90.20,9,85.19,0.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1112445,X6P111,PF00018,IPR001452,193,240,48,10,63,54,187,...,56,178,247,70,48,100.00,85.71,9,85.71,0.92
1112447,X6PEP5,PF14604,IPR001452,9,56,48,15,68,54,6,...,54,2,62,61,48,100.00,88.89,7,88.89,0.94
1112448,X7YG90,PF02775,IPR011766,422,575,154,2,16,15,507,...,15,507,521,15,15,9.74,100.00,140,9.74,0.18
1112451,X7ZSI2,PF02775,IPR011766,422,575,154,2,16,15,507,...,15,507,521,15,15,9.74,100.00,140,9.74,0.18
